In [1]:
import numpy as np
import pandas as pd

In [8]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [9]:
df_train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [10]:
df_train = df_train.drop(['PassengerId','Cabin','Name'],axis='columns')
df_test = df_test.drop(['PassengerId','Cabin','Name'],axis='columns')

In [11]:
df_train.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True


In [14]:
for idx,col in enumerate(df_train.columns):
    column_null_values=df_train[col].isnull().sum()
    if column_null_values>0:
        print('The number of nan values in column',col,'in index',idx,'is',column_null_values,df_train.dtypes[col])

The number of nan values in column HomePlanet in index 0 is 201 object
The number of nan values in column CryoSleep in index 1 is 217 object
The number of nan values in column Destination in index 2 is 182 object
The number of nan values in column Age in index 3 is 179 float64
The number of nan values in column VIP in index 4 is 203 object
The number of nan values in column RoomService in index 5 is 181 float64
The number of nan values in column FoodCourt in index 6 is 183 float64
The number of nan values in column ShoppingMall in index 7 is 208 float64
The number of nan values in column Spa in index 8 is 183 float64
The number of nan values in column VRDeck in index 9 is 188 float64


In [15]:
X_train = df_train.iloc[:,:-1].values
y_train = df_train.iloc[:,-1].values
X_test = df_test.iloc[:,:].values

In [16]:
from sklearn.impute import SimpleImputer
mean_imputer = SimpleImputer(missing_values = np.nan,strategy ='mean')
freq_imputer = SimpleImputer(missing_values = np.nan,strategy = 'most_frequent')
X_train[:,[3,5,6,7,8,9]] = mean_imputer.fit_transform(X_train[:,[3,5,6,7,8,9]])
X_train[:,[0,1,2,4]] = freq_imputer.fit_transform(X_train[:,[0,1,2,4]])
X_test[:,[3,5,6,7,8,9]] = mean_imputer.fit_transform(X_test[:,[3,5,6,7,8,9]])
X_test[:,[0,1,2,4]] = freq_imputer.fit_transform(X_test[:,[0,1,2,4]])

In [17]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[0,1,2,4])],remainder='passthrough')
le = LabelEncoder()
X_train = np.array(ct.fit_transform(X_train))
X_test = np.array(ct.transform(X_test))
y_train = le.fit_transform(y_train)
print(X_train)


[[0.0 1.0 0.0 ... 0.0 0.0 0.0]
 [1.0 0.0 0.0 ... 25.0 549.0 44.0]
 [0.0 1.0 0.0 ... 0.0 6715.0 49.0]
 ...
 [1.0 0.0 0.0 ... 1872.0 1.0 0.0]
 [0.0 1.0 0.0 ... 0.0 353.0 3235.0]
 [0.0 1.0 0.0 ... 0.0 0.0 12.0]]


In [19]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100,max_depth =5, criterion='entropy', random_state=0)
classifier.fit(X_train,y_train)


RandomForestClassifier(criterion='entropy', max_depth=5, random_state=0)

In [20]:
y_pred=le.inverse_transform(classifier.predict(X_test))

In [21]:
y_pred

array([ True, False,  True, ...,  True,  True,  True])